<a href="https://colab.research.google.com/github/Tverdostup1993/Python_for_ds_tasks/blob/main/%D0%A2%D0%B2%D0%B5%D1%80%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF_%D0%9E_%D0%86_%22HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("customer_segmentation_train.csv")
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
df.isnull().sum()

,0
ID,0
Gender,0
Ever_Married,140
Age,0
Graduated,78
Profession,124
Work_Experience,829
Spending_Score,0
Family_Size,335
Var_1,76


In [ ]:
# % пропущених значень по кожному стовпцю
missing_percent = df.isnull().mean().round(4) * 100

# Виводимо лише ті, де є пропуски
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)

# Вивід
print("Відсоток пропущених значень у кожному стовпці:")
print(missing_percent)


Відсоток пропущених значень у кожному стовпці:
Work_Experience    10.28
Family_Size         4.15
Ever_Married        1.74
Profession          1.54
Graduated           0.97
Var_1               0.94
dtype: float64


Замінюю числові дані медіаною, а категорійні модою, хоча ці дані які мають меньше 5% пропущених значень можна б було прибрати

In [ ]:
# числові змінні
num_cols = df.select_dtypes(include=['int64', 'float64']).columns

# категоріальні змінні
cat_cols = df.select_dtypes(include='object').columns.drop('Segmentation')

# заповнення числових NaN
for col in num_cols:
    df[col].fillna(df[col].median(), inplace=True)

# заповнення категоріальних NaN
for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)


/tmp/ipython-input-7-1834727750.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-7-1834727750.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

In [ ]:
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

In [ ]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop('Segmentation', axis=1)
y = df_encoded['Segmentation']

# розбиваємо (stratify — щоб зберегти пропорції класів)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
X_train

,ID,Age,Work_Experience,Family_Size,Gender_Male,Ever_Married_Yes,Graduated_Yes,Profession_Doctor,Profession_Engineer,Profession_Entertainment,...,Profession_Lawyer,Profession_Marketing,Spending_Score_High,Spending_Score_Low,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7
917,465905,32,9.0,1.0,False,False,True,False,False,False,...,False,False,False,True,False,False,False,False,True,False
3398,462903,72,1.0,2.0,True,True,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
2045,467901,33,1.0,4.0,False,False,True,False,False,True,...,False,False,False,True,False,False,False,False,True,False
8060,463613,48,0.0,6.0,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4604,459859,28,9.0,1.0,False,True,False,True,False,False,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3822,463101,27,8.0,1.0,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
5864,467844,37,0.0,2.0,True,False,True,False,False,False,...,False,False,False,True,False,False,False,False,True,False
3589,460706,27,6.0,6.0,False,False,False,False,True,False,...,False,False,False,True,False,False,True,False,False,False
1489,464339,26,0.0,2.0,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False


In [ ]:
y_train

,Segmentation
917,A
3398,B
2045,B
8060,A
4604,A
...,...
3822,D
5864,D
3589,D
1489,D


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
non_cat_features = [col for col in X_train.columns if not any(c in col for c in cat_cols)]
X_train_numeric = X_train[non_cat_features]

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Perform random sampling
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_numeric, y_train)

# Train linear SVC
#clf_smote = SVC(kernel='linear',probability=True)
#clf_smote.fit(X_train_smote, y_train_smote)

#clf_lr = LogisticRegression(max_iter=1000)
#clf_lr.fit(X_train_smote, y_train_smote)


In [ ]:
print("До SMOTE:", X_train_numeric.shape, y_train.value_counts().to_dict())
print("Після SMOTE:", X_train_smote.shape, y_train_smote.value_counts().to_dict())

До SMOTE: (6454, 4) {'D': 1814, 'A': 1578, 'C': 1576, 'B': 1486}
Після SMOTE: (7256, 4) {'A': 1814, 'B': 1814, 'C': 1814, 'D': 1814}


In [ ]:
from imblearn.over_sampling import SMOTENC

cat_features = [col for col in X_train.columns if any(c in col for c in cat_cols)]
cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features]

# SMOTENC
smotenc = SMOTENC(categorical_features=cat_feature_indices, random_state=42)
X_smotenc, y_smotenc = smotenc.fit_resample(X_train, y_train)

print("Після SMOTENC:\n", y_smotenc.value_counts())

Після SMOTENC:
 Segmentation
A    1814
B    1814
C    1814
D    1814
Name: count, dtype: int64


In [ ]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(smote=SMOTENC(categorical_features=cat_feature_indices, random_state=42), random_state=42)
X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(X_train, y_train)

print("Після SMOTE-Tomek:\n", y_smote_tomek.value_counts())

Після SMOTE-Tomek:
 Segmentation
C    1378
D    1369
B    1299
A    1260
Name: count, dtype: int64


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [ ]:
model_orig = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model_orig.fit(X_train, y_train)
y_pred_orig = model_orig.predict(X_test)
print(" Оригінальні дані")
print(classification_report(y_test, y_pred_orig))

 Оригінальні дані
              precision    recall  f1-score   support

           A       0.41      0.48      0.44       394
           B       0.39      0.16      0.23       372
           C       0.48      0.60      0.53       394
           D       0.65      0.73      0.69       454

    accuracy                           0.51      1614
   macro avg       0.48      0.49      0.47      1614
weighted avg       0.49      0.51      0.49      1614



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
model_smote = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model_smote.fit(X_train_smote, y_train_smote)
y_pred_smote = model_smote.predict(X_train_smote)
print("Після SMOTE")
print(classification_report(y_train_smote, y_pred_smote))

Після SMOTE
              precision    recall  f1-score   support

           A       0.34      0.31      0.33      1814
           B       0.30      0.11      0.16      1814
           C       0.39      0.46      0.42      1814
           D       0.47      0.72      0.57      1814

    accuracy                           0.40      7256
   macro avg       0.37      0.40      0.37      7256
weighted avg       0.37      0.40      0.37      7256



In [ ]:
# 3. Модель на SMOTE-Tomek-даних
model_tomek = OneVsRestClassifier(LogisticRegression(max_iter=1000))
model_tomek.fit(X_smote_tomek, y_smote_tomek)
y_pred_tomek = model_tomek.predict(X_smote_tomek)
print("Після SMOTE-Tomek")
print(classification_report(y_smote_tomek, y_pred_tomek))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Після SMOTE-Tomek
              precision    recall  f1-score   support

           A       0.45      0.51      0.48      1260
           B       0.43      0.24      0.31      1299
           C       0.52      0.64      0.58      1378
           D       0.67      0.74      0.70      1369

    accuracy                           0.54      5306
   macro avg       0.52      0.53      0.52      5306
weighted avg       0.52      0.54      0.52      5306



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Обрана метрика — f1_macro, оскільки вона враховує якість класифікації для кожного класу однаково, незалежно від розміру вибірки.

Найкращий результат показала модель після SMOTE-Tomek з f1_macro = 0.52.
Це пояснюється тим, що SMOTE-Tomek не лише балансує класи, а й очищає дані від прикордонного "шуму", покращуючи розділення між сегментами.

Усі три моделі показали приблизно однакову якість, з незначною перевагою у варіанті з SMOTE-Tomek.
Ймовірно, це пояснюється тим, що:

- Ознаки недостатньо інформативні для якісної класифікації

- Логістична регресія не здатна вловити складні залежності між ознаками

- SMOTE та SMOTE-Tomek працюють на базі вже обмеженої інформації та не додають "нової якості"